## pretty_midi

In [10]:
import pretty_midi
# Create a PrettyMIDI object
cello_c_chord = pretty_midi.PrettyMIDI()
# Create an Instrument instance for a cello instrument
cello_program = pretty_midi.instrument_name_to_program('Cello')
cello = pretty_midi.Instrument(program=cello_program)
# Iterate over note names, which will be converted to note number later
for note_name in ['C4', 'C4', 'G5','G5','A5','A5','G5',]:
    # Retrieve the MIDI note number for this note name
    note_number = pretty_midi.note_name_to_number(note_name)
    # Create a Note instance, starting at 0s and ending at .5s
    note = pretty_midi.Note(
        velocity=100, pitch=note_number, start=0, end=.5)
    # Add it to our cello instrument
    cello.notes.append(note)
# Add the cello instrument to the PrettyMIDI object
cello_c_chord.instruments.append(cello)
# Write out the MIDI data
cello_c_chord.write('cello-C-chord.mid')

In [11]:
import pretty_midi
# Load MIDI file into PrettyMIDI object
midi_data = pretty_midi.PrettyMIDI('cello-C-chord.mid')
# Print an empirical estimate of its global tempo
print(midi_data.estimate_tempo())
# Compute the relative amount of each semitone across the entire song, a proxy for key
total_velocity = sum(sum(midi_data.get_chroma()))
print([sum(semitone)/total_velocity for semitone in midi_data.get_chroma()])
# Shift all notes up by 5 semitones
for instrument in midi_data.instruments:
    # Don't want to shift drum notes
    if not instrument.is_drum:
        for note in instrument.notes:
            note.pitch += 5
# Synthesize the resulting MIDI data using sine waves
audio_data = midi_data.synthesize()
IPython.display.Audio(midi_data.synthesize(fs=16000), rate=16000)

ValueError: Can't provide a global tempo estimate when there are fewer than two notes.

## misic21

In [54]:
from music21 import note,stream,corpus,chord,environment,converter,midi

stream1 = stream.Stream()
note = note.Note("C4", quarterLength = 1)
stream1.repeatAppend(note, 4)
stream1.show("midi")

In [1]:
from music21 import note,stream,corpus,chord,environment,converter,midi

stream2 = stream.Stream()
n1 = note.Note("C4", quarterLength = 1)
n4 = note.Note("G4", quarterLength = 1)
noteList=[]

noteList.append(n1)
noteList.append(n1)
noteList.append(n4)

stream2.append(noteList)
stream2.show("midi")

StreamException: the object (<music21.note.Note C>, id()=4338933600) is already found in this Stream (<music21.stream.Stream 0x1029edf28>, id()=4338933544)

In [6]:
from music21 import note,stream,corpus,chord,environment,converter,midi
stream2 = converter.parse('tinyNotation: 4/4 c4 c4 g4 g4 a4 a4 g4')
print(stream2)
stream2.show("midi")

<music21.stream.Part 0x11300dd30>


In [1]:
from music21 import converter, instrument, note, chord, stream
import numpy as np
nocturne = converter.parse('twinkle_twinkle.mid')
nocturne.show("midi")

In [2]:
nocturne = converter.parse('twinkle_twinkle.mid')

parts = instrument.partitionByInstrument(nocturne)

print(parts)

notes_to_parse = parts.parts[0].recurse()  # 対象のパートを一つに絞る

print(notes_to_parse)

string_nocturne_notes = []

for element in notes_to_parse:
    if isinstance(element, note.Note):
      string_nocturne_notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
      string_nocturne_notes.append('.'.join(str(n) for n in element.normalOrder))

<music21.stream.Score 0x113974390>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>


In [3]:
print(string_nocturne_notes)

['C4', 'C3', 'C4', 'G4', 'E3', 'G4', 'A4', 'F3', 'A4', '4.7', 'F4', 'D3', 'F4', 'E4', 'C3', 'E4', '2.5', '2.7', 'C4', 'E3', 'C3', 'G4', 'E3', 'G4', 'F4', 'F3', 'F4', 'E4', 'G3', 'E4', '2.7', 'G4', 'E3', 'G4', 'F4', 'F3', 'F4', 'E4', 'G3', 'E4', '2.7', 'C4', 'C3', 'C4', 'G4', 'E3', 'G4', 'A4', 'F3', 'A4', '4.7', 'F4', 'D3', 'F4', 'E4', 'C3', 'E4', '2.5', '2.7', '0']


In [4]:
notenames = sorted(set(string_nocturne_notes)) #音名
print(notenames)

['0', '2.5', '2.7', '4.7', 'A4', 'C3', 'C4', 'D3', 'E3', 'E4', 'F3', 'F4', 'G3', 'G4']


In [5]:
note2int = dict((string_note, number) for number, string_note in enumerate(notenames))
print(note2int)

{'0': 0, '2.5': 1, '2.7': 2, '4.7': 3, 'A4': 4, 'C3': 5, 'C4': 6, 'D3': 7, 'E3': 8, 'E4': 9, 'F3': 10, 'F4': 11, 'G3': 12, 'G4': 13}


In [6]:
numerical_nocturne_notes = []
for string_note in string_nocturne_notes:
    numerical_nocturne_notes.append(note2int[string_note])
    
print(numerical_nocturne_notes)

[6, 5, 6, 13, 8, 13, 4, 10, 4, 3, 11, 7, 11, 9, 5, 9, 1, 2, 6, 8, 5, 13, 8, 13, 11, 10, 11, 9, 12, 9, 2, 13, 8, 13, 11, 10, 11, 9, 12, 9, 2, 6, 5, 6, 13, 8, 13, 4, 10, 4, 3, 11, 7, 11, 9, 5, 9, 1, 2, 0]


In [7]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

In [8]:
create_midi(numerical_nocturne_notes)

TypeError: argument of type 'int' is not iterable